In [0]:
#Pre-requisite: If the code is executed in Google Colab, download the dataset and upload it to your google drive in the same location
#Dataset: https://drive.google.com/file/d/1vKVt7sd2f8D5aSNxoggcdHbrThHxlXx7/view?usp=sharing
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#Required to run once per session.
pip install librosa --upgrade

In [3]:
## Implementation of GAN
#1. Importing required packages
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU, ReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import librosa
from librosa import display

#2. Constructing GAN class that contains the model for the discriminator and generator
class MusicGAN():
  def __init__(self):
    self.spect_freq = 128 
    self.spect_time = 640 
    self.channel = 1 
    self.img_shape = (self.spect_freq, self.spect_time, self.channel)
    self.noise_dimension = 64
    self.discriminator = self.build_discriminator()
    self.discriminator.compile(loss='binary_crossentropy',
                                optimizer=Adam(0.0002, 0.5),
                                metrics=['accuracy'])
    self.generator = self.build_generator()
    z = Input(shape=(self.noise_dimension,))
    img = self.generator(z)
    self.discriminator.trainable = False
    validity = self.discriminator(img)
    self.combined = Model(z, validity)
    self.combined.compile(loss='binary_crossentropy', optimizer= Adam(0.0001, 0.5))

  #3. Setting up generator model
  def build_generator(self):
      model = Sequential()
      momentum = 0.8
      model.add(Dense(256, input_dim=self.noise_dimension))
      model.add(LeakyReLU(alpha=0.2))
      model.add(BatchNormalization(momentum=momentum))
      model.add(Dense(512))
      model.add(LeakyReLU(alpha=0.2))
      model.add(BatchNormalization(momentum=momentum))
      model.add(Dense(1024))
      model.add(LeakyReLU(alpha=0.2))
      model.add(BatchNormalization(momentum=momentum))
      model.add(Dense(2048))
      model.add(LeakyReLU(alpha=0.2))
      model.add(BatchNormalization(momentum=momentum))
      model.add(Dense(4096))
      model.add(LeakyReLU(alpha=0.2))
      model.add(BatchNormalization(momentum=momentum))
      model.add(Dense(np.prod(self.img_shape), activation='tanh'))
      model.add(Reshape(self.img_shape))
      input_noise = Input(shape=(self.noise_dimension,))
      img = model(input_noise)
      model.summary()
      return Model(input_noise, img)

  #4. Setting up discriminator model
  def build_discriminator(self):
      model = Sequential()
      model.add(Flatten(input_shape=self.img_shape))
      model.add(Dense(4096))
      model.add(LeakyReLU(alpha=0.2))
      model.add(Dense(2048))
      model.add(LeakyReLU(alpha=0.2))
      model.add(Dense(1024))
      model.add(LeakyReLU(alpha=0.2))
      model.add(Dense(512))
      model.add(LeakyReLU(alpha=0.2))
      model.add(Dense(256))
      model.add(LeakyReLU(alpha=0.2))
      model.add(Dense(1, activation='sigmoid'))
      img = Input(shape=self.img_shape)
      validity = model(img)
      model.summary()
      return Model(img, validity)

  #5. Defining the training parameters
  def train(self, epochs, batch_size, sample_interval):
      '''
      Usage documentation can be found here: https://www.tensorflow.org/api_docs/python/tf/keras/datasets/mnist/load_data
      Since this is a unsupervised learning and we wanted only the training images, we ignored to load other x_test, y_train and y_test.
      '''
      train_data = np.load('/content/drive/My Drive/DL/GAN/shuffled_train.npz')
      self.x_train_rot = train_data['arr_0']
      size = 999
      self.x_train = np.empty((size,self.spect_freq,self.spect_time))
      for i in range(size):
          self.x_train[i,:,:] = np.rot90(self.x_train_rot[i,:,:],3)
      print(self.x_train.shape) 
      self.x_train = self.x_train.reshape(-1, self.spect_freq,self.spect_time, 1).astype(np.float32)
      
      
      print("input size")
      print(self.x_train.shape)
      valid = np.ones((batch_size, 1))
      fake = np.zeros((batch_size, 1))
      for epoch in range(epochs):
          idx = np.random.randint(0, self.x_train.shape[0], batch_size)
          imgs = self.x_train[idx]
          print("input training size")
          print(imgs.shape)
          input_noise = np.random.normal(0, 1, (batch_size, self.noise_dimension))
          g_loss = self.combined.train_on_batch(input_noise, valid)
          gen_imgs = self.generator.predict(input_noise)
          d_loss_real = self.discriminator.train_on_batch(imgs, valid)
          d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
          d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
          print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))
          if epoch % sample_interval == 0:
            self.generate_spectrogram(input_noise,epoch)

  #6. Generating new numbers at given sample interval		
  def generate_spectrogram(self, input_noise, epoch):
      predictions = gan.generator.predict(input_noise)
      print("generated output size")
      print(predictions.shape)
      print("Reshaping")
      generated_seq = np.reshape(predictions[0], (self.spect_freq,self.spect_time))
      print(generated_seq.shape)
      self.convert_to_audio(generated_seq, epoch)
      librosa.display.specshow(generated_seq, y_axis='mel', fmax=8000, x_axis='time')
      plt.colorbar(format='%+2.0f dB')
      plt.savefig("/content/drive/My Drive/DL/GAN/Results/GAN_MUSIC/GeneratedOutputForEpoch-%d.png" % epoch)
      plt.show()
      plt.close()
  #7. Generate audio sample from spectogram
  def convert_to_audio (self, dbspectogram, epoch):
      # db_to_power(S_db) ~= ref * 10.0**(S_db / 10)
      audio_spectogram = librosa.core.db_to_power(dbspectogram)
      #Converting spectogram into time series array
      audio_timeSeries = librosa.feature.inverse.mel_to_audio(audio_spectogram, sr = 22050, hop_length=1024)
      #Converting time series array into audio file
      audiofile_path = "/content/drive/My Drive/DL/GAN/Results/GAN_MUSIC/GeneratedMusicForEpoch-%d.wav" % epoch
      librosa.output.write_wav(audiofile_path, audio_timeSeries, sr=22050, norm=False)

Using TensorFlow backend.


In [0]:
#7. Running both the models with specified epochs, batch size and sample intervals (to less generated samples, the sample_interval size should be increased)
gan = MusicGAN()    
gan.train(epochs=10000, batch_size=32, sample_interval=50)

43 [D loss: 28.616009, acc.: 71.88%] [G loss: 40.393307]
input training size
(32, 128, 640, 1)
